In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# グラフの日本語表示設定
# plt.rcParams['font.sans-serif'] = ['Hiragino Sans'] # Macの場合
# # plt.rcParams['font.sans-serif'] = ['MS Gothic'] # Windowsの場合
# # plt.rcParams['font.family'] = 'IPAexGothic' # LinuxなどでIPAフォントをインストールした場合
# plt.rcParams['axes.unicode_minus'] = False # マイナス記号の表示設定

In [8]:
# ### 1. データの読み込みと前処理

df_raw = pd.read_csv('../data/visitors.csv')
print("--- df_raw.head() ---")
print(df_raw.head())
print("\n--- df_raw.info() ---")
df_raw.info()

# データには注釈行や国名が空欄の行、集計行が含まれているため、これらを処理します。

# 注釈行の削除 (country列が '注１' または '注２' で始まる行を除外)
df_cleaned = df_raw[~df_raw['country'].str.startswith('注１', na=False) & ~df_raw['country'].str.startswith('注２', na=False)].copy()

# visitors列を数値型に変換 (変換できないものはNaNに)
df_cleaned.loc[:, 'visitors'] = pd.to_numeric(df_cleaned['visitors'], errors='coerce')

# 国名がNaNまたは空欄の行を除外
df_cleaned.dropna(subset=['country'], inplace=True)
df_cleaned = df_cleaned[df_cleaned['country'].str.strip() != '']

# 集計行を除外 (例: '総数', '計' で終わるもの)
excluded_keywords = ['総数', '計', 'その他'] # 'その他' も広すぎるため一旦除外検討
pattern = '|'.join(excluded_keywords) # '総数' OR '計' OR 'その他'
df_cleaned = df_cleaned[~df_cleaned['country'].str.contains(pattern, na=False)]

print("\n--- df_cleaned.head() after exclusions ---")
print(df_cleaned.head())
print(f"除外処理後のデータ数: {len(df_cleaned)}")

# 次に、`year` と `month` から日付型の列を作成します。
# 月の表記が「1月」などになっているため、数値に変換します。

def convert_month_to_int(month_str):
    if isinstance(month_str, str):
        return int(month_str.replace('月', ''))
    return np.nan

df_cleaned.loc[:, 'month_num'] = df_cleaned['month'].apply(convert_month_to_int)

# year, month_num, day=1としてdatetimeオブジェクトを作成 (dayは仮に1日とする)
# 欠損値を含む行はここで除外されるか、NaTになる
df_cleaned.dropna(subset=['year', 'month_num'], inplace=True) # yearやmonth_numがNaNの行を除外
df_cleaned.loc[:, 'date'] = pd.to_datetime(df_cleaned['year'].astype(int).astype(str) + '-' +
                                     df_cleaned['month_num'].astype(int).astype(str) + '-' + '1', errors='coerce')

# 不要になったmonth_num列を削除、visitorsがNaNの行もこの段階で除去
df_cleaned.drop(columns=['month_num'], inplace=True)
df_cleaned.dropna(subset=['visitors', 'date'], inplace=True) # visitors や date がNaNの行を除外
df_cleaned.loc[:, 'visitors'] = df_cleaned['visitors'].astype(int) # visitorsを整数型に

print("\n--- df_cleaned.head() after date conversion ---")
print(df_cleaned.head())
print("\n--- df_cleaned.info() after date conversion ---")
df_cleaned.info() 

--- df_raw.head() ---
   year month country   visitors
0  2025    1月      総数  3781629.0
1  2025    1月    アジア計  3261149.0
2  2025    1月      韓国   967100.0
3  2025    1月      中国   980520.0
4  2025    1月      台湾   593431.0

--- df_raw.info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13716 entries, 0 to 13715
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      13716 non-null  int64  
 1   month     13716 non-null  object 
 2   country   13356 non-null  object 
 3   visitors  12632 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 428.8+ KB

--- df_cleaned.head() after exclusions ---
   year month country  visitors
2  2025    1月      韓国  967100.0
3  2025    1月      中国  980520.0
4  2025    1月      台湾  593431.0
5  2025    1月      香港  243687.0
6  2025    1月      タイ   96811.0
除外処理後のデータ数: 9996

--- df_cleaned.head() after date conversion ---
   year month country  visitors       date
2  202